In [6]:
import pandas as pd
from datasets import Dataset

In [8]:
train_df = pd.read_csv("../data/train.csv")
valid_df = pd.read_csv("../data/valid.csv")
train_df.head()

,Unnamed: 0,text,label
0,94625,The Maltese Falcon film noir based on the nove...,1
1,89585,Water is a scarce resource so when it is dispe...,1
2,187840,I was really expecting a lot given all the hyp...,1
3,265694,"This movie was just awful. It was not scary, t...",0
4,125407,"""The Town"" is an excellent film that does have...",0


In [10]:
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)
train_ds

Dataset({
    features: ['Unnamed: 0', 'text', 'label'],
    num_rows: 459130
})